In [1]:
import pandas as pd
import plotly.express as px

from dash import Dash, dcc, html, Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.tools as tls

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
crop_df = pd.read_csv('data/fao_processed_unstacked.csv')
crop_df.head()


,Unnamed: 0,year,area,item,Area harvested,Production,Yield,Country,Alpha-3 code,Code
0,2,1961,Austria,Asparagus,35.0,150.0,42857.0,Austria,AUT,AUT
1,3,1961,Austria,Barley,187648.0,511571.0,27262.0,Austria,AUT,AUT
2,4,1961,Austria,"Beans, dry",682.0,1160.0,17009.0,Austria,AUT,AUT
3,6,1961,Austria,"Broad beans and horse beans, dry",659.0,1635.0,24810.0,Austria,AUT,AUT
4,8,1961,Austria,Buckwheat,224.0,256.0,11429.0,Austria,AUT,AUT


In [3]:
fao_raw_percentiles_df = pd.read_csv('data/fao_raw_percentiles.csv')

In [4]:
PERCENTILE_LABELS = ["0-25%", "25-50%", "50-75%", "75-100%"]

In [8]:
app = Dash(__name__)

app.layout = html.Div([
    html.Div([   
        dcc.Dropdown(
            id="crop",
            options=crop_df.item.unique(),
            value="Barley",
        ),
        dcc.Dropdown(
            id="metric",
            options=['Area Harvested', 'Production', 'Yield'],
            value="Production",
        ),
        dcc.Graph(id="crop-eu-map"),
    ], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([   
        dcc.Dropdown(
            id="crop",
            options=crop_df.item.unique(),
            value="Barley",
        ),
        dcc.Dropdown(
            id="metric",
            options=['Area Harvested', 'Production', 'Yield'],
            value="Production",
        ),
        dcc.Graph(id="crop-eu-map2"),
    ], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        html.Div([
            html.Div([
                dcc.Dropdown(
                    id="area",
                    options=fao_raw_percentiles_df.area.unique(),
                    value="Ireland",
                ),
                dcc.RadioItems(
                    PERCENTILE_LABELS,
                    PERCENTILE_LABELS[-1],
                    id="percentile",
                    inline=True,
                )
            ], style={'width': '40%', 'display': 'inline-block'}),
        ]),
        dcc.Graph(id="line-graph"),
        dcc.Graph(id="bar-graph"),
    ], style={'width': '99%', 'display': 'inline-block'}),
], style={'background': 'white', 'padding': '2px'})


@app.callback(
    [
    Output("crop-eu-map", "figure"), 
    Output("crop-eu-map2", "figure")],
    Input("crop", "value"),
    Input("metric", "value"))
def update_map(crop, metric):
    data = crop_df[crop_df.item == crop]
   
    fig = px.choropleth(data,
        locations='Code',
        color=metric,
        animation_frame="year",
        hover_name='area'
    )
    
    fig.update_layout(
        title=dict(
            text='Crops',
            x=.5,
            font_size=18,
            ),
    
        geo=dict(bgcolor='#8ad6ff', lakecolor='#8ad6ff', projection_type='miller', scope='europe'),
        
        width = 500,
        height = 500,
        coloraxis=dict(colorscale="Reds", cmin=data['Yield'].min(), cmax=data['Yield'].max())
    )
    return fig, fig

@app.callback([
    Output("line-graph", "figure"), 
    Output("bar-graph", "figure")], 
    Input("percentile", "value"),
    Input("area", "value"))
def update_line_chart(percentile, area):
    data=fao_raw_percentiles_df[(fao_raw_percentiles_df.value > 0.0) & (fao_raw_percentiles_df.unit == 't') & (fao_raw_percentiles_df.area == area) & (fao_raw_percentiles_df.percentile == percentile)]
    return (px.line(data, x="year", y="value", color='item', title=f"{percentile} Crops Production in {area}"), 
            px.bar(data, x="year", y="value", color="item", title=f"{percentile} Crops Production in {area}"))


app.run_server(debug=True, port=8077)

---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `crop`

